In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install python_speech_features librosa matplotlib playsound PyObjC

In [2]:
from os import listdir
from os.path import isdir, join
import librosa
import random
import numpy as np
import matplotlib.pyplot as plt
import python_speech_features

In [3]:
# Dataset path and view possible targets
dataset_path = '/Users/jona/Documents/Github/Community/python-examples/python_examples/models/data_speech_commands_v0.02'
for name in listdir(dataset_path):
    if isdir(join(dataset_path, name)):
        pass
        # print(name)

In [8]:
# Create an all targets list
all_targets = [name for name in listdir(dataset_path) if isdir(join(dataset_path, name))]
print(all_targets)

['right', 'eight', 'cat', 'tree', 'backward', 'learn', 'bed', 'happy', 'go', 'dog', 'no', 'wow', 'follow', 'nine', 'left', 'stop', 'three', '_background_noise_', 'sheila', 'one', 'bird', 'zero', 'seven', 'up', 'visual', 'marvin', 'two', 'house', 'down', 'six', 'yes', 'on', 'five', 'forward', 'off', 'four']


In [9]:
# Leave off background noise set
all_targets.remove('_background_noise_')
print(all_targets)

['right', 'eight', 'cat', 'tree', 'backward', 'learn', 'bed', 'happy', 'go', 'dog', 'no', 'wow', 'follow', 'nine', 'left', 'stop', 'three', 'sheila', 'one', 'bird', 'zero', 'seven', 'up', 'visual', 'marvin', 'two', 'house', 'down', 'six', 'yes', 'on', 'five', 'forward', 'off', 'four']


In [10]:
# See how many files are in each
num_samples = 0
for target in all_targets:
    # print(len(listdir(join(dataset_path, target))))
    num_samples += len(listdir(join(dataset_path, target)))
print('Total samples:', num_samples)

Total samples: 105829


In [73]:
# Settings
target_list = all_targets
feature_sets_file = 'all_targets_mfcc_sets.npz'
perc_keep_samples = 1.0 # 1.0 is keep all samples
val_ratio = 0.1
test_ratio = 0.1
sample_rate = 8000
num_mfcc = 16
len_mfcc = 16

In [80]:
# Create list of filenames along with ground truth vector (y)
filenames = []
absolute_path = []
y = []
for index, target in enumerate(target_list):
    # print(join(dataset_path, target))
    filesn = listdir(join(dataset_path, target))
    filenames.append(filesn)
    files = [{
        "id": x,
        "is_hotword": 1 if target == 'bird' else 0, 
        "worker_id": 12, 
        "duration": 1.86, 
        "audio_file_path": join(dataset_path, target, x), 
    } for x in filesn]
    absolute_path.extend(files)
    y.append(np.ones(len(filenames[index])) * index)

# print(absolute_path)

In [81]:
# Calculate validation and test set sizes
absolute_path = absolute_path[:int(len(absolute_path) * 0.1)]

val_set_size = int(len(absolute_path) * val_ratio)
test_set_size = int(len(absolute_path) * test_ratio)
# Break dataset apart into train, validation, and test sets
absolute_path_dev = absolute_path[:val_set_size]
absolute_path_test = absolute_path[val_set_size:(val_set_size + test_set_size)]
absolute_path_train = absolute_path[(val_set_size + test_set_size):]

In [82]:
import json
with open('train.json', 'w') as json_file:
    json.dump(absolute_path_train, json_file)
with open('test.json', 'w') as json_file:
    json.dump(absolute_path_test, json_file)
with open('dev.json', 'w') as json_file:
    json.dump(absolute_path_dev, json_file)